# Object Detection YoloV5

Muhammad Harits Shofwan Adani
1203210003

## 1. Datasets
Dataset yang digunakan terdapat 3 class yaitu: Shark, Ship, Submarine.
Pemilihan 3 class ini digunakan untuk mengetahui seberapa cerdas program dalam mendeteksi objek beserta relasi antar class.

### 1.1. Shark - Submarine
2 Class ini memiliki relasi yaitu berada di dalam air.

### 1.2. Ship - Submarine
2 Class ini memiliki relasi yaitu berupa sebuah kendaraan yang berada di permukaan air. Hal ini dikarenakan kapal selam memiliki dua kondisi yaitu berada di dalam air dan di permukaan air.

---

## 2. Create Folder Data

DataV1 nantinya akan menyimpan data images dan labels dengan masing-masing 3 sub-folder yaitu train (untuk training), val (untuk validasi), test (untuk tesgting).

In [6]:
import os

if not os.path.exists('DataV1'):
    for folder in ['images', 'labels']:
        for split in ['train', 'val', 'test']:
            os.makedirs(f'DataV1/{folder}/{split}')

### 2.1. Get File

Membuat variabel gambar yang didapat dari path folder images masing-masing class

In [3]:
import glob
import numpy as np
import shutil
def get_filenames(folder):
    filenames = set()
    allowed_extensions = ('.jpeg', '.jpg', '.png')

    for ext in allowed_extensions:
        for path in glob.glob(os.path.join(folder, f'*{ext}')):
            # Extract the filename
            filename = os.path.split(path)[-1]
            filenames.add(filename)

    return filenames


# Shark, Ship, and Submarine
shark_images = get_filenames('datasets/shark/images')
ship_images = get_filenames('datasets/ship/images')
submarine_images = get_filenames('datasets/submarine/images')

### 2.2. Split Image & Label

Variabel images akan di random yang kemudian akan dimasukkan ke dalam tiap folder yaitu train, val, test.

In [4]:
shark_images = list(shark_images)
ship_images = list(ship_images)
submarine_images = list(submarine_images)

# Use the same random seed for reproducability
np.random.seed(42)
np.random.shuffle(shark_images)
np.random.shuffle(ship_images)
np.random.shuffle(submarine_images)

In [7]:
def split_dataset(category, image_names, train_ratio, val_ratio):
    total_images = len(image_names)
    train_size = int(total_images * train_ratio)
    val_size = int(total_images * val_ratio)

    for i, image_name in enumerate(image_names):
        # Label filename
        base_name, ext = os.path.splitext(image_name)
        label_name = base_name + '.txt'
        
        # Split into train or test
        if i < train_size:
            split = 'train'
        elif i < train_size + val_size:
            split = 'val'
        else:
            split = 'test'
        
        # Source paths
        source_image_path = os.path.join('datasets', category, 'images', image_name)
        source_label_path = os.path.join('datasets', category, 'labels', label_name)

        # Destination paths
        target_image_folder = os.path.join('DataV1/images', split)
        target_label_folder = os.path.join('DataV1/labels', split)

        # Copy files
        shutil.copy(source_image_path, target_image_folder)
        shutil.copy(source_label_path, target_label_folder)

split_dataset('shark', shark_images, train_ratio=0.8, val_ratio=0.1)
split_dataset('ship', ship_images, train_ratio=0.8, val_ratio=0.1)
split_dataset('submarine', submarine_images, train_ratio=0.8, val_ratio=0.1)

## 3. Transfer Learning & Training Model
Menjalankan training pada model menggunakan YoloV5 train.py dengan data dari 3 folder yaitu data training, data validation, dan data test yang berasal dari 3 class yaitu Shark, Ship, dan Submarine. Weight data pretrained dan epoch sebanyak 200 dan batch sebanyak 4.

In [13]:
!python yolov5/train.py --data shark_ship_submarine.yaml --weights yolov5s.pt --epochs 200 --batch 4 --freeze 10 

train: weights=yolov5s.pt, cfg=, data=shark_ship_submarine.yaml, hyp=yolov5\data\hyps\hyp.scratch-low.yaml, epochs=200, batch_size=4, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5\runs\train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[10], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 
fatal: cannot change to 'E:\Semester': No such file or directory
YOLOv5  2023-11-7 Python-3.11.5 torch-2.1.0+cpu CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor

train/exp5 : epoch 200 batch 4

train/exp4 : epoch 50 batch 4

## 5. Prediction
Prediksi menggunakan YoloV5 detect.py dengan weights berasal dari model yang sudah dilatih, menghasilkan output dengan lebar 650px dengan tingkat confidence sebesar 0.25, setelah itu cantumkan source image/video yang akan di deteksi.

In [19]:
!python yolov5/detect.py --weights yolov5/runs/train/exp5/weights/last.pt --img 640 --conf 0.25 --source testing/KRI405.png

detect: weights=['yolov5/runs/train/exp5/weights/last.pt'], source=testing/KRI405.png, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
fatal: cannot change to 'E:\Semester': No such file or directory
YOLOv5  2023-11-7 Python-3.11.5 torch-2.1.0+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs
image 1/1 E:\Semester 5\Visi Komputer\Week 6\1203210003_VisKom_Object-Detection\testing\KRI405.png: 384x640 1 submarine, 178.6ms
Speed: 5.9ms pre-process, 178.6ms inference, 0.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to yolov5\runs\detect\exp7
